In [142]:
import requests
import pandas as pd
import re
import pickle
import os

In [42]:
url_tpl = "https://unsplash.com/napi/search/photos?query=full%20hd&xp=&per_page=300&page={}"
directory = "photo"

# Find image URLs

In [80]:
page = 1
all_photo_df = pd.DataFrame()

while True:
    url = url_tpl.format(page)
    resp = requests.get(url = url)
    photo_list = resp.json()["results"]
    if len(photo_list) == 0:
        break
    photo_df = pd.DataFrame(photo_list)
    all_photo_df = pd.concat([all_photo_df, photo_df])
    page = page + 1
    
all_photo_df = all_photo_df.reset_index(drop=True)

In [140]:
all_photo_df_mini = pd.DataFrame({
    'width': all_photo_df['width'],
    'height': all_photo_df['height'],
    'url': all_photo_df['urls'].apply(lambda x : x['raw'])
})

In [148]:
all_photo_file = open('all_photo_df.pkl', 'wb')
pickle.dump(all_photo_df_mini, all_photo_file)
all_photo_file.close()

# Download images

In [ ]:
num = 0
location_tpl = "../images/photo_raw/%04d_%04d_%04d.jpg"

for id, photo in all_photo_df_mini.iterrows():
    width = photo['width']
    height = photo['height']
    im_url = photo['url']
    location = location_tpl % (num, width, height)
    if not os.path.isfile(location):
        img = requests.get(im_url).content
        with open(location, 'wb') as f:
            f.write(img)
    num = num + 1

# Convert images